In [1]:
import os
import datetime
import urllib.request
import zipfile
import warnings
warnings.filterwarnings('ignore')

from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib import request
from urllib.parse import quote

import pickle
from tqdm import tqdm
import pandas as pd

In [2]:
class Loader:
    
    def __init__(self, date_start, date_end, period, region='ru'): 
        # check start and end date, start date must be less than end day, end date <= today
        if date_end > (datetime.datetime.date(datetime.datetime.now()) - datetime.timedelta(days=2)).strftime('%Y-%m-%d'):
            date_end = (datetime.datetime.date(datetime.datetime.now()) - datetime.timedelta(days=2)).strftime('%Y-%m-%d')
        if date_start >= date_end: date_end = date_start
             
        self.date_start = self.formate(date_start)
        self.date_end = self.formate(date_end) 
        self.period = period + '/' 
        self.region = region + '-'
        #self.destinationPath = 'C:\Python\sber\pet_project'
        self.destinationPath = './'
    
    def formate(self, date):
        self.year, self.month, self.day = map(int, date.split('-'))
        return datetime.date(self.year, self.month, self.day) 
    
    def make_url(self):
        diff = int((self.date_end - self.date_start).days)
        for days in tqdm(range(diff)):
            today_str = (self.date_end - datetime.timedelta(days=days)).strftime('%Y-%m-%d')
            # read data archive https://mediametrics.ru/data/archive/
            file_name = self.region + today_str + '.zip'
            url = 'http://mediametrics.ru/data/archive/' + self.period + file_name 
            try: # Download the file from `url` and save it locally under `file_name`:
                urllib.request.urlretrieve(url, file_name)
                sourceZip = zipfile.ZipFile(file_name, 'r')
                # print(url, days)
            except (ValueError, TimeoutError):
                pass
            for name in sourceZip.namelist():
                sourceZip.extract(name, self.destinationPath)
            sourceZip.close()
            os.remove(file_name) # remove download files

In [3]:
%%time
# date_start = input('Enter a start date in YYYY-MM-DD format')
# date_end = input('Enter a end date in YYYY-MM-DD format')
if __name__ == '__main__':
    period = input('Period - day or hour or month or online or week :')
    if period == '':
        period = 'week'
    #region = input('region')
    url = 'http://mediametrics.ru/data/archive/' + period
    html = urlopen(url)
    bsObj = BeautifulSoup(html.read(), "lxml")
    titles = bsObj.find_all('pre')
    m = str(titles).split('</a>')
    date = [m[i][-14:-4] for i in range(len(m)) if m[i].endswith('.zip')][:-1] # find start date and end date
    
    date_start = date[-2]
    date_end = date[-1]
    date = Loader(date_start, date_end, period)
    date.make_url()

Period - day or hour or month or online or week :


100%|██████████| 1/1 [00:33<00:00, 33.49s/it]


Wall time: 35 s
